In [ ]:

!pip -q install torch torchvision torchaudio --upgrade
!pip -q install scikit-learn matplotlib pillow tqdm


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

DATASET_DIR = "/content/drive/MyDrive/chest_xray"

import os, glob, random, shutil
assert os.path.exists(DATASET_DIR), f"Path not found: {DATASET_DIR}"
print("Drive dataset root:", DATASET_DIR)
ROOT = DATASET_DIR

In [ ]:
import os, glob, random, shutil

def has_split(root):
    return all(os.path.isdir(os.path.join(root, s)) for s in ["train","val","test"])

if not has_split(ROOT):
    if all(os.path.isdir(os.path.join(ROOT, s)) for s in ["train","test"]):
        val_dir = os.path.join(ROOT, "val")
        os.makedirs(val_dir, exist_ok=True)
        for cls in os.listdir(os.path.join(ROOT, "train")):
            src = os.path.join(ROOT, "train", cls)
            dst = os.path.join(val_dir, cls)
            os.makedirs(dst, exist_ok=True)
            files = [f for f in glob.glob(os.path.join(src, "*")) if os.path.isfile(f)]
            random.shuffle(files)
            take = max(1, int(0.1*len(files)))
            for f in files[:take]:
                shutil.copy2(f, os.path.join(dst, os.path.basename(f)))
        print("Created validation split on Drive (copied ~10% from train).")
    else:

        classes = [d for d in os.listdir(ROOT) if os.path.isdir(os.path.join(ROOT, d))]
        if classes:
            print("Creating 80/10/10 split from flat class folders on Drive (copying files)...")
            split_root = os.path.join(ROOT, "_split")
            for split in ["train","val","test"]:
                for cls in classes:
                    os.makedirs(os.path.join(split_root, split, cls), exist_ok=True)
            for cls in classes:
                files = [f for f in glob.glob(os.path.join(ROOT, cls, "*")) if os.path.isfile(f)]
                random.shuffle(files)
                n = len(files); n_train = int(0.8*n); n_val = int(0.1*n)
                for i,f in enumerate(files):
                    if i < n_train:
                        dst = os.path.join(split_root, "train", cls, os.path.basename(f))
                    elif i < n_train + n_val:
                        dst = os.path.join(split_root, "val", cls, os.path.basename(f))
                    else:
                        dst = os.path.join(split_root, "test", cls, os.path.basename(f))
                    shutil.copy2(f, dst)
            ROOT = split_root

print("Using dataset root with splits:", ROOT)
!find "$ROOT" -maxdepth 2 -type d -print


In [ ]:

import torch, os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

IMG_SIZE = 224
BATCH_SIZE = 32

train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])
val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

train_ds = datasets.ImageFolder(os.path.join(ROOT, "train"), transform=train_tfms)
val_ds   = datasets.ImageFolder(os.path.join(ROOT, "val"),   transform=val_tfms)
test_ds  = datasets.ImageFolder(os.path.join(ROOT, "test"),  transform=val_tfms)

num_workers = 0
pin_memory = False

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=num_workers, pin_memory=pin_memory)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=num_workers, pin_memory=pin_memory)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=num_workers, pin_memory=pin_memory)

class_names = train_ds.classes
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
len(train_ds), len(val_ds), len(test_ds), class_names, DEVICE


In [ ]:

import torch
from collections import Counter

targets = [y for _, y in train_ds.samples]
counts = Counter(targets)
num_classes = len(class_names)
total = sum(counts.values())
class_weights = torch.zeros(num_classes, dtype=torch.float)
for c in range(num_classes):
    class_weights[c] = total / (num_classes * counts[c])
print("Class counts:", counts)
print("Class weights:", class_weights)


In [ ]:

import torch, torch.nn as nn, torch.optim as optim
from torchvision import models
from tqdm.auto import tqdm
import os

PROJECT_NAME = "pneumonia_vgg_colab"
DRIVE_DIR = f"/content/drive/MyDrive/{PROJECT_NAME}"
os.makedirs(DRIVE_DIR, exist_ok=True)

ARCH = "vgg16"
LR = 1e-3
WD = 1e-4
FREEZE_FEATURES_EPOCHS = 1
EPOCHS = 8

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

if ARCH == "vgg16":
    model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
else:
    model = models.vgg19(weights=models.VGG19_Weights.DEFAULT)

in_features = model.classifier[-1].in_features
model.classifier[-1] = nn.Linear(in_features, len(class_names))
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss(weight=class_weights.to(DEVICE))
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

if FREEZE_FEATURES_EPOCHS > 0:
    for p in model.features.parameters():
        p.requires_grad = False
print("Features frozen for first", FREEZE_FEATURES_EPOCHS, "epochs")

scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))

best_val_acc = 0.0
history = {"train_loss":[], "val_acc":[]}

for epoch in range(1, EPOCHS+1):
    model.train()
    running = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS} - train"):
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
            outputs = model(images)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        running += loss.item() * images.size(0)
    train_loss = running / len(train_loader.dataset)

    if epoch == FREEZE_FEATURES_EPOCHS:
        for p in model.parameters(): p.requires_grad = True
        print("Unfroze all layers.")

    model.eval()
    correct = 0; total = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch}/{EPOCHS} - val"):
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
                outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    val_acc = correct / total if total>0 else 0.0
    scheduler.step(val_acc)

    history["train_loss"].append(train_loss)
    history["val_acc"].append(val_acc)
    print(f"Epoch {epoch}: train_loss={train_loss:.4f} val_acc={val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), os.path.join(DRIVE_DIR, f"best_{ARCH}.pth"))
        print("Saved new best model to Drive.")

print("Best val acc:", best_val_acc)


In [ ]:

import matplotlib.pyplot as plt, numpy as np, torch, os
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import itertools

plt.figure(); plt.plot(history["train_loss"], label="train_loss")
plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title("Training Loss"); plt.legend()
plt.savefig(os.path.join(DRIVE_DIR,"training_loss.png")); plt.show()

plt.figure(); plt.plot(history["val_acc"], label="val_acc")
plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.title("Validation Accuracy"); plt.legend()
plt.savefig(os.path.join(DRIVE_DIR,"val_accuracy.png")); plt.show()

state_dict_path = os.path.join(DRIVE_DIR, f"best_{ARCH}.pth")
model.load_state_dict(torch.load(state_dict_path, map_location=DEVICE))
model.eval()

all_labels, all_preds, all_probs = [], [], []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(DEVICE)
        with torch.cuda.amp.autocast(enabled=(DEVICE=='cuda')):
            outputs = model(images)
        probs = torch.softmax(outputs, dim=1).cpu().numpy()
        preds = np.argmax(probs, axis=1)
        all_labels.extend(labels.numpy())
        all_preds.extend(preds)
        if len(class_names)==2:
            all_probs.extend(probs[:,1])

print("Classes:", class_names)
print(classification_report(all_labels, all_preds, target_names=class_names))

cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\\n", cm)

plt.figure()
plt.imshow(cm, interpolation='nearest'); plt.title('Confusion matrix'); plt.colorbar()
ticks = np.arange(len(class_names))
plt.xticks(ticks, class_names, rotation=45); plt.yticks(ticks, class_names)
thr = cm.max()/2.
for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, f"{cm[i,j]}", ha="center", va="center",
             color="white" if cm[i,j] > thr else "black")
plt.ylabel('True label'); plt.xlabel('Predicted label'); plt.tight_layout()
plt.savefig(os.path.join(DRIVE_DIR,"confusion_matrix.png")); plt.show()

if len(class_names)==2 and len(all_probs)==len(all_labels):
    auc = roc_auc_score(all_labels, all_probs)
    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    print("ROC AUC:", auc)
    plt.figure(); plt.plot(fpr, tpr, label=f"AUC = {auc:.3f}"); plt.plot([0,1],[0,1],'--')
    plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("ROC Curve"); plt.legend(loc="lower right")
    plt.savefig(os.path.join(DRIVE_DIR,"roc_curve.png")); plt.show()
